# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: DL Modell erstellen

Hier in diesem Notebook erstellen wir unsere Deep Learning Modelle.

# Libraries Importieren & GPU Check

In [ ]:
# General Libraries
import logging
from datetime import datetime
import os

# Data Science Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn Libraries
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection as ms
from sklearn.model_selection import KFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score

# keras Libraries
from keras.utils import to_categorical

# Tensorflow Libraries
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

# dataclass
import dataclasses
from dataclasses import dataclass

# general setup
tf.debugging.set_log_device_placement(False)

# datetime as filename for logging
now = datetime.now()
date_time_string = now.strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(
    level=logging.INFO, filename=f"{date_time_string}.txt", filemode="a"
)
print(tf.config.list_physical_devices())

# Num of Gpus
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Daten einlesen

In [ ]:
# Static Parameters
@dataclass
class Parameters:
    batch_size: int = 128
    epochs: int = 15
    verbosity: str = "auto"
    number_folds: int = 2
    output_size: int = 6
    window_size: int = 300
    step_size: int = 100

# Getrimmte Sensordaten einlesen
df = pd.read_csv("../Sensor_Data-Wrangling-und-EDA/Alle_Messungen_trimmed.csv", index_col=0)
display(df)

# convert the string time column to datetime
epoch = pd.Timestamp("1970-01-01")

# Time column into datetime
df["time"] = pd.to_datetime(df["time"])

# Time column into milliseconds
df["time"] = (df["time"] - epoch).apply(lambda x: int(x.total_seconds() * 1000))

# Preprocessing Data

## Create Validation Data

In [ ]:
# select three random id_combines files
ids = ["06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren", 
       "01_iPhone13pro-2023-03-21_16-55-47Etienne_RouletLaufen", 
       "01_iPhone13ProMax-2023-03-15_18-29-42Gabriel_TorresRennen"]

# create a dataframe with these three random files
df_validation = df[df["id_combined"].isin(ids)]

# export the dataframe as a csv file
df_validation.to_csv("validation-velo-laufen-rennen.csv", index=False)

# remove the validation data from the dataframe 
df = df[~df["id_combined"].isin(ids)]

## Drop unnötige Spalten und Class Encoden

In [ ]:
# drop the columns that are not needed
df = df.drop(columns=["id", "user", "id_combined"])
# get all types of the df
le = LabelEncoder()
# encode the classes
df["class"] = le.fit_transform(df["class"])
# print dictionary of the classes and its encoded values
print(dict(zip(le.classes_, le.transform(le.classes_))))

## Daten Transformation

In [ ]:
# Set the window size and step size
window_size = Parameters.window_size
step_size = Parameters.step_size

# Reshape X to 2D format (samples, features)
X = df.values[:, 1:13]

# Define y
y = df["class"].values

# Create a sliding window of X with the specified window and step sizes
X_windows = np.array(
    [
        X[i : i + window_size, :]
        for i in range(0, X.shape[0] - window_size + 1, step_size)
    ]
)

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

# Create the corresponding y labels for the sliding windows
y_windows = np.array(
    [y[i + window_size - 1] for i in range(0, X.shape[0] - window_size + 1, step_size)]
)
y_windows = to_categorical(y_windows, num_classes=6)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    X_windows, y_windows, test_size=0.1, random_state=42, stratify=y_windows
)

# print Data shapes
print("Data shapes:")
print("- X:         = {}".format(X.shape))
print("- y:         = {}".format(y.shape))
print("- X_train:   = {}".format(x_train.shape))
print("- y_train:   = {}".format(y_train.shape))
print("- X_test:    = {}".format(x_test.shape))
print("- y_test:    = {}".format(y_test.shape))

# DL-Modelle erstellen

In [ ]:
# Creating Deep Learning Models

def create_model_1(name="model_1"):
    '''
    CNN Model with 1 Convolutional Layer, 1 LSTM Layer and 1 Dense Layer 
    '''
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=12,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_2(name="model_2"):
    '''
    CNN Model with one Conv1D layer, one LSTM layer and one Dense layer, also using l2 regularization and dropout
    '''
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=8,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.001),
            ),
            # Add LSTM layer
            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200)),
            tf.keras.layers.Dropout(0.5),

            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_3(name="model_3"):
    '''
    CNN Model with 32 Filters combine with LSTM and one dense layer
    '''
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_4(name="model_4"):
    '''
    CNN Model with 64 Filters combined with LSTM and one dense layer
    '''
    model = tf.keras.Sequential(
        [
            # Add a 1D convolutional layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            # Add LSTM layer
            tf.keras.layers.LSTM(100),
            # Add a dense output layer
            tf.keras.layers.Dense(
                6, activation="softmax"
            ),  # Change activation function based on the nature of the output
        ],
        name=name,
    )
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_5(name="model_5"):
    '''
    CNN Model with 3 Conv1D Layers and 2 Dense Layers with Batch Normalization and L2 Regularization
    '''
    model = tf.keras.Sequential(
        [
            # First Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=32,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Second Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=20, 
                kernel_size=2, 
                activation="relu", 
                padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Third Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=10, 
                kernel_size=2, 
                activation="relu", 
                padding="same", 
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Flatten the results
            tf.keras.layers.Flatten(),
            # Add first dense layer
            tf.keras.layers.Dense(units = 200, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Add second dense layer
            tf.keras.layers.Dense(units = 100, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Add output Layer with Softmax Activation Funktion
            tf.keras.layers.Dense(units = 6, 
                                  activation="softmax"),
        ],
        name=name,
    )
    # Define optimizer, loss and metrics 
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_6(name="model_6"):
    '''
    CNN Model with 2 Conv1D Layers and 3 Dense Layers with Batch Normalization and L2 Regularization
    '''
    model = tf.keras.Sequential(
        [
            # First Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=64,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Second Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=32, 
                kernel_size=2, 
                activation="relu", 
                padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Flatten the results
            tf.keras.layers.Flatten(),
            # Add first dense layer
            tf.keras.layers.Dense(units = 200, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Add second dense layer
            tf.keras.layers.Dense(units = 100, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Add third dense layer
            tf.keras.layers.Dense(units = 50, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(0.01),),
            tf.keras.layers.BatchNormalization(),
            # Add output Layer with Softmax Activation Funktion
            tf.keras.layers.Dense(units = 6, 
                                  activation="softmax"),
        ],
        name=name,
    )
    # Define optimizer, loss and metrics 
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model


def create_model_7(name="model_7", 
                   filterconv1 = 64, 
                   filterconv2 = 64, 
                   filterconv3 = 64, 
                   filterconv4 = 64,
                   filterconv5 = 64, 
                   unitsdense1 = 200, 
                   unitsdense2 = 150, 
                   unitsdense3 = 100,
                   unitsdense4 = 50,
                   unitsdense5 = 25,
                   l2_reg = 0.05):
    '''
    Check, if the deeper the model the better the results? 
    CNN Model with 5 Conv1D Layers and 5 Dense Layers with Batch Normalization and L2 Regularization
    '''
    model = tf.keras.Sequential(
        [
            # First Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=filterconv1,
                kernel_size=2,
                activation="relu",
                padding="same",
                input_shape=(timesteps, n_features),
                kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
            ),
            # Batchnormalization and Maxpooling
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Second Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=filterconv2, 
                kernel_size=2, 
                activation="relu", 
                padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
            ),
            # Batchnormalization and Maxpooling
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Third Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=filterconv3, 
                kernel_size=2, 
                activation="relu", 
                padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
            ),
            # Batchnormalization and Maxpooling
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Fourth Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=filterconv4,
                kernel_size=2,
                activation="relu",
                padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
            ),
            # Batchnormalization and Maxpooling
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Fifth Conv1D Layer
            tf.keras.layers.Conv1D(
                filters=filterconv5,
                kernel_size=2,
                activation="relu",
                padding="same",
                kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
            ),
            # Batchnormalization and Maxpooling
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            # Flatten the results
            tf.keras.layers.Flatten(),
            # Add first dense layer
            tf.keras.layers.Dense(units = unitsdense1, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),),
            tf.keras.layers.BatchNormalization(),
            # Add second dense layer
            tf.keras.layers.Dense(units = unitsdense2, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),),
            tf.keras.layers.BatchNormalization(),
            # Add third dense layer
            tf.keras.layers.Dense(units = unitsdense3, 
                                  activation="relu", 
                                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),),
            tf.keras.layers.BatchNormalization(),
            # Add fourth dense layer
            tf.keras.layers.Dense(units = unitsdense4,
                                  activation="relu",
                                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),),
            tf.keras.layers.BatchNormalization(),
            # add fifth dense layer
            tf.keras.layers.Dense(units = unitsdense5,
                                  activation="relu",
                                  kernel_regularizer=tf.keras.regularizers.l2(l2_reg),),
            tf.keras.layers.BatchNormalization(),
        
            # Add output Layer with Softmax Activation Funktion
            tf.keras.layers.Dense(units = 6, 
                                  activation="softmax"),
        ],
        name=name,
    )
    # Define optimizer, loss and metrics 
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
    )
    return model

# DL-Modelle Trainieren

In [ ]:
best_model_history = None  # Keep track of the best model's history
model_histories = []
models = [create_model_6] # Add all models to this list
best_model = None
num_folds = Parameters.number_folds
tscv = TimeSeriesSplit(n_splits=num_folds)
fold_acc_scores = []

for i, (train, test) in enumerate(tscv.split(x_train)):
    logging.info(f"Fold {i+1}")
    train_x, train_y = x_train[train], y_train[train]
    test_x, test_y = x_train[test], y_train[test]

    fold_histories = []

    for j, model_creator in enumerate(models):
        print(f"Model {model_creator.__name__}")
        model_name = f"Model_{j+1}_Fold_{i+1}"
        model = model_creator(name=model_name)
        logging.info(f"Model {j+1}")
        history = model.fit(
            train_x,
            train_y,
            epochs=Parameters.epochs,
            batch_size=Parameters.batch_size,
            validation_data=(test_x, test_y),
            verbose=Parameters.verbosity,
        )
        test_loss, acc, prec, recal = model.evaluate(
            test_x, test_y, verbose=Parameters.verbosity
        )
        logging.info(f"Validation accuracy: {acc}")

        fold_histories.append(history.history)

        for epoch in range(Parameters.epochs):
            # Log accuracy after each epoch
            acc_epoch = history.history["val_accuracy"][epoch]
            logging.info(f"Epoch {epoch + 1}, Validation accuracy: {acc_epoch}")
        fold_acc_scores.append((i, j, acc))

        if best_model_history is None or acc > best_model_acc:
            best_model_history = history
            best_model = model  # Store the trained model instance
            best_model_acc = acc

    model_histories.append(fold_histories)

# Find the best model
best_model_index = np.argmax([score[2] for score in fold_acc_scores])
best_fold_idx, best_model_idx, _ = max(fold_acc_scores, key=lambda x: x[2])
best_model_history = model_histories[best_fold_idx][best_model_idx]
print(best_model.name)

## Verlauf von Accuracy und Loss

In [ ]:
# Create 2 subplots one for model accuracy and another for model loss
fig, axs = plt.subplots(2, figsize=(15, 10))
# Summarize history for accuracy
axs[0].plot(best_model_history["accuracy"])
axs[0].plot(best_model_history["val_accuracy"])
axs[0].set_title("Model Accuracy")
axs[0].set_ylabel("Accuracy")
axs[0].set_xlabel("Epoch")
axs[0].legend(["train", "test"], loc="upper left")
# Summarize history for loss
axs[1].plot(best_model_history["loss"])
axs[1].plot(best_model_history["val_loss"])
axs[1].set_title("Model Loss")
axs[1].set_ylabel("Loss")
axs[1].set_xlabel("Epoch")
axs[1].legend(["train", "test"], loc="upper left")
plt.tight_layout()
plt.show()


## Confusions Matrix

In [ ]:
# Create for x_train and x_test confusion matrix
y_pred = best_model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_true = y_test_labels, y_pred = y_pred_labels)

cm = confusion_matrix(y_true = y_test_labels, y_pred = y_pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)

# Create subplot for test confusion matrix
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7.5))
disp.plot(cmap=plt.cm.Blues, ax=ax1, colorbar=False)
ax1.set_title("Test Confusion Matrix \n \
                Accuracy: {:.2f}%, Datapoints: {}".format(accuracy * 100, x_test.shape[0]), fontsize=18)
ax1.set_xticklabels(le.classes_, rotation=45)

# Create for x_train and x_test confusion matrix
y_pred = best_model.predict(x_train)
y_pred_labels = np.argmax(y_pred, axis=1)
y_train_labels = np.argmax(y_train, axis=1)

accuracy = accuracy_score(y_true = y_train_labels, y_pred = y_pred_labels)
cm = confusion_matrix(y_true = y_train_labels, y_pred = y_pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)

# Create subplot for train confusion matrix
disp.plot(cmap=plt.cm.Blues, ax=ax2, colorbar=False)
# add to title number of observations
ax2.set_title("Train Confusion Matrix \n \
              Accuracy: {:.2f}%, Datapoints: {}".format(accuracy * 100, x_train.shape[0]), fontsize=18)

ax2.set_xticklabels(le.classes_, rotation=45)

plt.tight_layout()
plt.show()


# Export DL-Modell

In [ ]:
# export Model to json Tensorflow file
import json

best_model.save("saved_model/sensor_model.h5")

model = tf.keras.models.load_model("saved_model/sensor_model.h5")

# Save model architecture to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# Save model weights to a JSON file
weights = model.get_weights()
weights_as_list = [w.tolist() for w in weights]
with open("weights.json", "w") as f:
    json.dump(weights_as_list, f)

# Modell Einlesen und predicten auf Testdaten

In [ ]:

class validate_unseen_data():
    def __init__(self, model_path="saved_model/sensor_model.h5"):
        self.model = tf.keras.models.load_model(model_path)
        self.csv_path = "validation-velo-laufen-rennen.csv"
        
    def predict_classes(self, file="06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren", expected=''):
        df_val = pd.read_csv(self.csv_path)
        # Filter the rows where its velo in id_combined
        df_val = df_val[df_val["id_combined"].str.contains(file)]
        df_val = df_val.drop(columns=["id", "user", "id_combined"])
        # convert the string time column to datetime
        epoch = pd.Timestamp("1970-01-01")
        df_val["time"] = pd.to_datetime(df_val["time"])
        df_val["time"] = (df_val["time"] - epoch).apply(
            lambda x: int(x.total_seconds() * 1000)
        )
        # get all types of the df
        le = LabelEncoder()
        df_val["class"] = le.fit_transform(df_val["class"])

        # Set the window size and step size
        window_size = 300
        step_size = 100

        # Reshape X to 2D format (samples, features)
        X = df_val.values[:, 1:13]
        y = df["class"].values

        # Create a sliding window of X with the specified window and step sizes
        X_windows = np.array(
            [
                X[i : i + window_size, :]
                for i in range(0, X.shape[0] - window_size + 1, step_size)
            ]
        )

        # Reshape X_windows to 3D format (samples, timesteps, features)
        timesteps = X_windows.shape[1]
        n_features = X_windows.shape[2]
        X_windows = X_windows.reshape(-1, timesteps, n_features)

        # Create the corresponding y labels for the sliding windows
        y_windows = np.array(
            [y[i + window_size - 1] for i in range(0, X.shape[0] - window_size + 1, step_size)]
        )
        y_windows = to_categorical(y_windows, num_classes=6)

        # predict validation data
        y_pred_probs = model.predict(X_windows)

        # Get the predicted class labels for each input window
        y_pred_labels = np.argmax(y_pred_probs, axis=1)

        # Print the predicted class labels
        # get the median of the predicted labels
        sol = np.median(y_pred_labels)

        class_counts = np.bincount(y_pred_labels)
        for i, count in enumerate(class_counts):
            print(f"Class {i} count: {count}")
        
        return (sol, expected)

median = validate_unseen_data().predict_classes('06_iPhone12-2023-03-16_13-46-58Manjavy_KirupaVelofahren', expected='velo')
median2 = validate_unseen_data().predict_classes('01_iPhone13ProMax-2023-03-15_18-29-42Gabriel_TorresRennen', expected='rennen')
median3 = validate_unseen_data().predict_classes('01_iPhone13pro-2023-03-21_16-55-47Etienne_RouletLaufen', expected='laufen')

print((median, median2, median3))



print(dict(zip(le.classes_, le.transform(le.classes_))))

In [ ]:
# the Conclusion is that the model is not overfitted but Gabriel is more a Stepper than a Runner ;)